In [ ]:
import pandas as pd
import numpy as np

# IMPORT DATA

In [ ]:
df = pd.read_excel('data_pointage_2022-12-07.xlsx')

### DATA COUNT

In [ ]:
print(df.shape)

### DATA TYPE

In [ ]:
print(df.dtypes)

# DATA PREPROCESSING

### SUPPRESSION DE COLUMNS INUTILE

In [ ]:
df.head()

In [ ]:
df.drop(['id', 'matiere', 'etudiant'],axis=1, inplace=True)
df.head()

### GERER LES VALEURS MANQUANTES

In [ ]:
data_null = df.isnull()
data_null

In [ ]:
df.dropna(axis=0, inplace=True)
df.isnull()

### NORMALISATION DU DATASET

CHANGEMENT DU VALEUR CATEGORQUE EN NUMERIQUE

In [ ]:
score = {
    "YES": 1,
    "NO": 0
    }
df['pointageIn'] = df['pointageIn'].map(score)
df['pointageOut'] = df['pointageOut'].map(score)
df['pointage'] = df.apply(lambda x: x['pointageIn'] + x['pointageOut'], axis=1)
df.drop(['pointageIn', 'pointageOut'], axis=1, inplace=True)
df.dtypes

GENERALISATION DU DATETIME (SPLIT DATE AND TIME)

In [ ]:
from datetime import datetime
def get_date(d):
    deb = datetime.strptime(d, '%Y-%m-%dT%H:%M:%SZ')
    return deb.date()
def get_time(d):
    deb = datetime.strptime(d, '%Y-%m-%dT%H:%M:%SZ')
    return deb.time()
    
df['date'] = df.apply(lambda x: get_date(x['datetimeDebut']), axis=1)
df['time'] = df.apply(lambda x: get_time(x['datetimeDebut']), axis=1)
df.dtypes

CALCUL DE LA DIFFERENCE ENTRE 2 DATETIME

In [ ]:
def diff_datetime(d, f):
    deb = datetime.strptime(d, '%Y-%m-%dT%H:%M:%SZ')
    fin = datetime.strptime(f, '%Y-%m-%dT%H:%M:%SZ')
    diff = fin - deb
    return diff.total_seconds()//60
df['duree_mn'] = df.apply(lambda x: diff_datetime(x['datetimeDebut'], x['datetimeFin']), axis=1)
df

In [ ]:
df.drop(['datetimeDebut', 'datetimeFin'], axis=1, inplace=True)
df

NORMALISE LES NOM DE COLONNES

In [ ]:
df.rename(columns={'nom_etudiant': 'Etudiant', 'nom_matiere': 'Matiere', 'nom_parcours': 'Parcours', 'nom_niveau': 'Niveau', 'nom_profs': "Enseignant", 'sex_etudiant': 'Sex', 'pointage': 'Pointage', 'date': 'Date', 'time': 'Time', 'duree_mn': 'Duree_mn'}, inplace=True)
df

In [ ]:
df.sort_values('Date')
df

# ANALYSE GLOBAL

In [ ]:
import matplotlib.pyplot as plt

### REPARTITION DES ACTIVITES PAR MATIERES (GENERAL)

In [ ]:
df['Matiere'].value_counts().plot.pie()
plt.show()

### REPATITION DES ACTIVITES PAR ENSEIGNANT

In [ ]:
df['Enseignant'].value_counts().plot.pie()
plt.show()

### AGILITE DES ETUDIANTS

In [ ]:
df['Pointage'].value_counts().plot.pie()
plt.show()

# ANALYSE PROFOND

## ANALYSE DES DONNEES ETUDIANTS

### GROUPEE PAR GENRE

In [ ]:
group_by_sex = df.groupby(['Niveau', 'Parcours', 'Sex', 'Pointage'])['Etudiant']
percent_gbs = ((group_by_sex.count()/group_by_sex.count().groupby(level=[0, 1, 2]).sum())*100).unstack().fillna(0)
percent_gbs

In [ ]:
percent_gbs.plot.barh()
plt.show()

CONCLUSION <br>
Les filles du L1 GB sont les plus actives <br>
Il y a quelques etudiants qui manquent l'un des deux marquage pour L1/IG (En retard, ou sortie en avant la fin du cours) <br>

## ANALYSE PROFOND PAR MATIERE

In [ ]:
group_by_matiere = df.groupby(['Matiere', 'Date', 'Time', 'Niveau', 'Parcours'])['Duree_mn']
data_matiere = {'Matiere': [], 'Duree': [], 'Niveau': [], 'Parcours': []}
for mat, frame in group_by_matiere:
    data_matiere['Matiere'].append(mat[0])
    data_matiere['Niveau'].append(mat[3])
    data_matiere['Parcours'].append(mat[4])
    data_matiere['Duree'].append(frame.mean())
df_matiere = pd.DataFrame(data_matiere)
df_matiere.sort_values(by=['Matiere', 'Niveau', 'Parcours'])[['Matiere', 'Niveau', 'Parcours']]
df_matiere.head()

### DUREE TOTAL D'HEURE PAR CHAQUE MATIERE

In [ ]:
group_by_mat = df_matiere.groupby(['Matiere'])
group_by_mat.sum()/60

In [ ]:
(group_by_mat.sum()/60).plot.barh()

CONCLUSION <br>
La matiere 'LANGAGE DE PROGRAMMATION JAVA' a le nombre d'heures le plus elevEs (plus de 30H) <br>
Par contre la matiere 'Protocole TCP/IP' n'a que moins de 8H de temps 

### NOMBRE DES COURS PRIS PAR CHAQUE NIVEAU/PARCOURS

In [ ]:
group_by_mat_PN = df_matiere.groupby(['Matiere', 'Niveau', 'Parcours'])
group_by_mat_PN.count()

In [ ]:
group_by_mat_PN.count().unstack().fillna(0).plot.barh()

CONCLUSION <br>
D'apres cet graphe, on peut voir tout de suite qu'il y a eu une erreur lors de la creation de ce donnees (donnees fake), pour le cours 'Systeme d'exploitation Unix' pour le L1/GB<br>
D'autre part, on peut en deduire que la matiere 'LANGAGE DE PROGRAMMATION JAVA' pour le GB a la valeur plus elevEs<br>
Et aussi, les etudiants du L1/GB ont pris un plus de cours d'ANGLAIS que L1/IG 

### DUREE TOTAL D'HEURE D'ETUDE PAR CHAQUE PARCOURS/NIVEAU

In [ ]:
group_by_PN = df_matiere.groupby(['Niveau', 'Parcours'])['Duree']
group_by_PN.sum()/60

In [ ]:
(group_by_PN.sum()/60).plot.barh()

CONCLUSION <br>
Les etudiants L1/GB ont plus de 100H de cours theorique, par contre L1/IG n'ont que soixantaine d'heures 

## ANALYSE PROFOND DU POINTAGE

In [ ]:
group_by_pointage = df.groupby(['Matiere', 'Date', 'Time', 'Niveau', 'Parcours', 'Pointage'])['Etudiant']
data_pointage = {'Matiere': [], 'Date': [], 'Time': [], 'Niveau': [], 'Parcours': [], 'Pointage': [], 'Etudiant': []}
for point, frame in group_by_pointage:
    data_pointage['Matiere'].append(point[0])
    data_pointage['Date'].append(point[1])
    data_pointage['Time'].append(point[2])
    data_pointage['Niveau'].append(point[3])
    data_pointage['Parcours'].append(point[4])
    data_pointage['Pointage'].append(point[5])
    data_pointage['Etudiant'].append(frame.count())
df_pointage = pd.DataFrame(data_pointage)
print(df_pointage.shape)
df_pointage.head()

In [ ]:
df_pointage.sort_values(by=['Time'])[['Time']]
pointage_group_by_time = df_pointage.groupby(['Time', 'Pointage'])['Etudiant']
pointage_group_by_time.sum()

In [ ]:
percent = (pointage_group_by_time.sum() / pointage_group_by_time.sum().groupby(level=[0]).sum())*100
percent

In [ ]:
percent.unstack().fillna(0).plot.barh()

CONCLUSION <br>
On peut en deduire a partir de cet graphe que l'agilitE des etudiants le matin et l'apres midi n'ont pas beaucoup de difference <br>
Mais par contre a 07:30, il y a quelques etudiant qui manquent une marquarge, Probablement en retard

### VARIATION DU POINTAGE AU FIL DU TEMPS

In [ ]:
df_pointage.sort_values(by=['Date', 'Time', 'Matiere'])[['Matiere', 'Date', 'Time']]
pointage_group_by_MDTP = df_pointage.groupby(['Date', 'Time', 'Matiere', 'Pointage'])['Etudiant']
print(pointage_group_by_MDTP.sum().shape)
pointage_group_by_MDTP.sum()
percent_MDTP = (pointage_group_by_MDTP.sum() / pointage_group_by_MDTP.sum().groupby(level=[0, 1, 2]).sum())*100
percent_MDTP.unstack().fillna(0).reset_index(level=[1], drop=True).reset_index(level=[1])

In [ ]:
percent_MDTP.unstack().fillna(0).reset_index(level=[1], drop=True).reset_index(level=[1]).plot.bar(figsize=(22, 10))

In [ ]:
df_pointage.sort_values(by=['Date', 'Time', 'Matiere'])[['Matiere', 'Date', 'Time']]
pointage_group_by_MDTP = df_pointage.groupby(['Date', 'Time', 'Matiere', 'Pointage'])['Etudiant']
print(pointage_group_by_MDTP.sum().shape)
pointage_group_by_MDTP.sum()
percent_MDTP = (pointage_group_by_MDTP.sum() / pointage_group_by_MDTP.sum().groupby(level=[0, 1, 2]).sum())*100
percent_MDTP.unstack().fillna(0).reset_index(level=[1], drop=True).reset_index(level=[1])